### Import and set-up

In [5]:
from dataclasses import dataclass, field
from typing import List, Tuple, Any

In [ ]:
'''
This module creates a .
It includes functions for calculating squares, cubes, and factorials.


-Naming conventions for operations
    - Suffix = addressing mode (I, _ABS, _IDX, _IDX_OFF)
        - I = immediate
        - _ABS operates on F[ABS]
        - _IDX operates on F[IDX]
        - _IDX_OFF operates on F[IDX + OFF] 
    Verb = operation (LOAD, STORE, ADD, JUMP)

- Helper functions in step() work on:
    - no helper function: LOADI, STOREI
    - _get_address(): LOAD_ABS, STORE_ABS, ADD_ABS
    - _check_idx(): LOAD_IDX, STORE_IDX, ADD_IDX
    - _get_target_index(): INC_IDX, DEC_IDX, LOAD_IDX_OFF, STORE_IDX_OFF, ADD_IDX_OFF, LOADI_IDX
    - _get_target_step(): JUMP_ABS, JUMP_REL
    
To-do new ops:
    [] If requires args[0], add to list "needs_args"
    [] Amend a helper function to validate things that can go wrong, or create new helper function if needed
    [] Note: this vm only accepts (op: str, int) or (op, ) as instruction
 
'''

Instruction = Tuple[str, Any]

# create boilerplate for dataclass CPU
# dataclass decorator creates class boilerplates that already have __init__, __repr__, and __eq__ defined
@dataclass
class CPU:
    mem: List[int] = field(default_factory=lambda: [0]*32)      # 32 integer cells of RAM
    acc: int = 0                                                # Accumulator
    idx: int = 0                                                # Index register
    pc: int = 0                                                 # Program counter
    halted: bool = False
    program: List[Instruction] = field(default_factory=list)    # program is a List of Instructions (list of tuples)

    # load program, with starting values for each field except mem
    def load_program(self, program: List[Instruction]):
        '''Load a list of instructions and reset CPU state.'''
        self.program = program
        self.pc = 0
        self.halted = False
        self.acc = 0
        self.idx = 0
        # memory persists until you overwrite cpu.mem explicitly

    def dump(self):
        '''Snapshot of key state for quick inspection'''
        return{
            'PC': self.pc,
            'ACC': self.acc,
            'IDX': self.idx,
            'HALTED': self.halted,
            'MEMO..7': self.mem[:8]
        }
    
    def step(self):
        '''Execute exactly the next instruction'''
        # if halted or pc is not between 0 and len(program), set halted = True
        if self.halted or not (0 <= self.pc < len(self.program)):
            self.halted = True 
            return
                
        # set-up
        instr = self.program[self.pc]
        # unpack instruction into op and *args or 
        op, *args = instr if isinstance(instr, tuple) else (instr,)

        # default: advance PC before execute (pre-increment)
        self.pc += 1

        # reset variables
        target_index = None
        addr = None
        target_step = None
        
        # enforce that *args[0] is int
        if len(args) >= 1 and not isinstance(args[0], int):
            raise TypeError(f'({op}, {args}): second tuple item must be an int')
        
        # enforce that certain ops have args[0]
        # note: 
        needs_args = {
            'LOADI', 'LOAD_ABS', 'LOAD_IDX_OFF',
            'STORE_ABS','STORE_IDX_OFF',
            'ADDI', 'ADD_ABS', 'ADD_IDX_OFF'
            'JUMP_ABS', 'JUMP_REL', 'LOADI_IDX',
            }
        if op in needs_args:
            if not args:
                raise ValueError(f'Instruction {self.pc} must include an int.')

        
        # HELPER FUNCTIONS
        # use with any command that operates on an ABS memory address
        def _get_address(args: Any) -> int:
            addr = args[0]
            if not (0 <= addr < len(self.mem)):
                raise IndexError(f'Instruction {self.pc} results in a bad memory address in step: {addr}.')
            return addr

        # use with Any command that operates on the IDX
        def _get_target_idx(op: str, args: Any) -> int:
            if 'OFF' in op: 
                off = args[0]
                target_idx = self.idx + off
            elif op == 'DEC_IDX':
                target_idx = self.idx - 1
            elif 'INC' in op:
                target_idx = self.idx + 1
            elif op == 'LOADI_IDX':
                target_idx = args[0]

            if not 0 <= target_idx < len(self.mem):
                raise IndexError(f'Instruction {self.pc} results in IDX out of range: {target_idx}.')
            
            return target_idx 

        # use with any command that operates on the memory slot @IDX
        def _check_idx():
            if not (0 <= self.idx < len(self.mem)):
                raise IndexError(f'Instruction {self.pc} contains an IDX out of range.')
            
        # use with any command that operates on self.pc
        def _get_target_step(args: Any) -> int:
            if op == 'JUMP_REL':
                target_step = self.pc + args[0]
            else:                       # includes JUMP_ABS
                target_step = args[0]
            if not (0 <= target_step < len(self.program)):
                raise IndexError(f'Instruction {self.pc} results in a bad pc: {target_step}')
            return target_step

        # OPERATION LOGIC
        # no helper function
        if op == 'LOADI':           # ACC <- immediate
            self.acc = args[0]
        elif op == 'ADDI':          # acc = acc + immediate
            self.acc = self.acc + args[0]

        # _get_address() helper function
        elif op == 'LOAD_ABS':      # ACC <- mem[addr]
            addr = _get_address(args)
            self.acc = self.mem[addr]
        elif op == 'STORE_ABS':     # mem[addr] <- ACC
            addr = _get_address(args)
            self.mem[addr] = self.acc
        elif op == 'ADD_ABS':       # acc = acc + mem[addr]
            addr = _get_address(args)
            self.acc = self.acc + self.mem[addr]

        # _check_id() helper function
        elif op == 'LOAD_IDX':      # ACC <- mem[idx]
            _check_idx()
            self.acc = self.mem[self.idx]
        elif op == 'STORE_IDX':     # mem[idx] <- ACC
            _check_idx()
            self.mem[self.idx] = self.acc
        elif op == 'ADD_IDX':       # acc = acc + mem[idx]
            _check_idx()
            self.acc += self.mem[self.idx]

        # _get_target_idx() helper function
        elif op == 'LOAD_IDX_OFF':  # ACC <- mem[idx+off]
            target_index = _get_target_idx(self, op, args)
            self.acc = self.mem[target_index]
        elif op == 'STORE_IDX_OFF':  # mem[idx+off] <- ACC
            target_index = _get_target_idx(op, args)
            self.mem[target_index] = self.acc
        elif op == 'ADD_IDX_OFF':   # acc = acc + mem[idx + off]
            target_index = _get_target_idx(op, args)
            self.acc += self.mem[target_index]
        elif op == 'INC_IDX':
            target_index = _get_target_idx(op, args)     
            self.idx = target_index
        elif op == 'DEC_IDX':
            target_index = _get_target_idx(op, args)     
            self.idx = target_index
        elif op == 'LOADI_IDX':       # idx = args[0]
            target_index = _get_target_idx(op, args)
            self.idx = target_index
        
        # get_target_set() helper function
        elif op == 'JUMP_ABS':      # Absolute jump
            target_step = _get_target_step(args)
            self.pc = target_step
        elif op == 'JUMP_REL':      # Relative jump
            target_step = _get_target_step(args)
            self.pc = target_step
        elif op == 'HALT':          # stop execution
            self.halted = True
        else:
            raise NotImplementedError(f'Unknown op: {op}')
        
    def run(self, max_steps=100):
        '''Run up to max_steps or until halted.'''
        steps = 0
        while not self.halted and steps < max_steps:
            self.step()
            steps += 1
        return steps


In [ ]:
prog = [
    ('LOADI', -7),
    ('STORE_ABS', 0),
    ('LOADI', 12),
    ('STORE_ABS', 1),
    ('LOAD_ABS', 1),
    ('LOADI_IDX', 5),
    ('HALT',),
]

In [141]:
cpu = CPU()
cpu.load_program(prog)
cpu.run()
print(cpu.dump())


{'PC': 7, 'ACC': 12, 'IDX': 5, 'HALTED': True, 'MEMO..7': [-7, 12, 0, 0, 0, 0, 0, 0]}
